In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

### TODO: find and use panama dataset, and ETTh1 for comparing metrics

In [23]:
# X_1d = torch.tensor((pd.read_csv("electricity.csv").to_numpy())) # yet to verify if this is how the model expects the data.
X_1d = torch.rand(10, 3)
print(f'{X_1d.shape = }')

X_1d.shape = torch.Size([10, 3])


### hyperparameters

In [24]:
k = 6       # TODO: find out how the default top k value is selected.
T = 96      # sequence length (?)
H = 48      # forecast horizon (?)

### FFT

In [25]:
def fft_period(x_1d:torch.Tensor, k:int=5):
    """
    Extract top-k periods from 1d time series using FFT.
    
    args:
        x_1d: torch.Tensor of shape [T,C], where T is no. of timesteps, c is no. of channels
        k: number of top frequencies to extract (default=5)
    
    returns:
        A: amplitude values for all frequencies
        freqs: top-k frequency indices
        periods: corresponding period lengths
    """
    
    T, C = x_1d.shape
    
    A = torch.abs(torch.fft.rfft(x_1d)) # get amplitudes (TODO: verify wrt original implementation)
    A = torch.mean(A, dim=1) # mean along channel dimension; [T,1] dimensions
    A = A[:T/2] # only use first half due to conjugacy of freq domain

    freqs = torch.topk(A, k) # we only do it till T/2 due to conjugacy of frequency domain (TODO: understand why)
    periods = torch.ceil(T / freqs.float()).int() # CORRECTED
    
    return A, freqs, periods

### Padding before reshape

In [26]:
def zero_padding(x_1d:torch.Tensor, period: int):
    """
    Zeros added to the front.
    
    args:
        x_1d: torch.Tensor of shape [T, C]
        period: int, 
    """
    
    T, C = x_1d.shape
    
    remaining_length = T - T // period * period

    if remaining_length == 0:
        return x_1d
    else:
        return torch.cat([torch.zeros(period - remaining_length, C), x_1d], dim=0) # 0 = time dimension

In [27]:
def circular_padding(x_1d:torch.Tensor, period: int):
    """
    Stuff from back added to the front, in reverse order.
    
    args:
        x_1d: torch.Tensor of shape [T, C]
        period: int, 
    """
    
    T, C = x_1d.shape
    
    remaining_length = T - T // period * period

    if remaining_length == 0:
        return x_1d
    else:
        return torch.cat([x_1d[-remaining_length:,:][::-1,:], x_1d], dim=0) # 0 = time dimension

### Reshape to 2D

In [ ]:
x_2d_list = []

_, freqs, periods =  fft_period(X_1d, k=k)

for i in range(k):
    freq, period = freqs[i], periods[i]
    padded_list_2d = torch.reshape(zero_padding(X_1d, period), [period, freq])
    x_2d_list.append(padded_list_2d)

### TimesBlock

In [ ]:
A_prev, freqs, periods = 

In [ ]:
sample_t = 12
sample_c = 3

mean_amps = torch.mean(torch.abs(torch.fft.rfft(torch.rand([sample_t, sample_c]))), dim=1)
print(f"{mean_amps = }")

mean_amps = tensor([1.2689, 1.0900, 1.4195, 1.1849, 0.9105, 0.9596, 1.3262, 1.2720, 0.8588,
        1.1586, 1.2602, 1.2626])


In [ ]:
torch.topk(mean_amps[:], k=3)

torch.return_types.topk(
values=tensor([1.9793, 1.8895, 1.7872]),
indices=tensor([5, 3, 7]))